# Projeto 2 - Ciência dos Dados

Nome:Luís Filipe Loureiro

Nome: João Bueno


___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
# !pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

___
## Autenticando no  Twitter

* Conta: @luisfloureiro1

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @luisfloureiro1

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'Golf'

#Quantidade mínima de mensagens capturadas:
n = 1600
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []

for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break


#Embaralhando as mensagens para reduzir um possível viés
shuffle(list(set(msgs)))



Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(list(set(msgs))[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(list(set(msgs))[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [3]:
#Para não ser mais necessário conectar-se ao twitter, basta rodas essa cédula e a cédula que importa as bibliotecas
tweets= pd.read_excel('Golf.xlsx')


In [4]:
valores=[]

for valor in tweets.loc[:,"Relevancia"]:
    valores.append(valor)


In [5]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;/@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [6]:
clean_treinamento=[]
for tweet in tweets.loc[:,"Treinamento"]:
    clean_treinamento.append((cleanup(tweet)))

    
        
# clean_treinamento

In [7]:
tweets_clean =pd.DataFrame(clean_treinamento)
tweets_clean.loc[:,"Relevancia"]=1

In [8]:
#Adicionando a coluna relevancia
for numero in np.arange(0,500):
    tweets_clean.loc[numero,"Relevancia"]=valores[numero]

tweets_clean.columns=["tweet","Relevancia"]


In [9]:
#Retirando double spaces
for numero in np.arange(0,500):
    tweets_clean.loc[numero,"tweet"]=re.sub(' +', ' ',tweets_clean.loc[numero,"tweet"])
    tweets_clean.loc[numero,"tweet"]=tweets_clean.loc[numero,"tweet"].strip()


In [44]:
irrelevantes=tweets_clean.loc[tweets_clean.Relevancia==0]

neutros=tweets_clean.loc[tweets_clean.Relevancia==1]

relevantes=tweets_clean.loc[tweets_clean.Relevancia==2]

In [65]:
N=[]
for neutro in neutros.tweet:
    N.append(neutro)

N01=','.join(N)
serie_neutro=pd.Series(N01.split())
tabela_neutro=serie_neutro.value_counts()
tabela_neutro_relativa = serie_neutro.value_counts(True)
tabela_neutro_relativa.sum()

0.9999999999999998

In [66]:
I=[]
for irrelevante in irrelevantes.tweet:
    I.append(irrelevante)
    
I01=','.join(I)
serie_irrelevante=pd.Series(I01.split())
tabela_irrelevante=serie_irrelevante.value_counts()
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
tabela_irrelevante_relativa.sum()

1.0

In [67]:
R=[]
for relevante in relevantes.tweet:
    R.append(relevante)
    
R01=','.join(R)
serie_relevante=pd.Series(R01.split())
tabela_relevante=serie_relevante.value_counts()
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa.sum()

1.0

In [68]:
total=N01+I01+R01

In [69]:
serie_Total=pd.Series(total.split())

In [70]:
tabela_Total_relativa = serie_Total.value_counts(True)

In [71]:
set_neutro= set(tabela_neutro_relativa.index)
set_irrelevante= set(tabela_irrelevante_relativa.index)
set_relevante= set(tabela_relevante_relativa.index)
int = set_neutro.intersection(set_irrelevante)

In [72]:
inter=set_relevante.intersection(int)
#inter

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [73]:
frase='muito bom o golf novo'
frase=frase.lower()
frase=cleanup(frase.lower())
frase=pd.Series(frase.split())
lista_frase=list(frase)
x=1
y=1
z=1


for palavra in lista_frase:
    p=(lista_frase.count(palavra))/(len(lista_frase))
    if palavra not in tabela_neutro_relativa or palavra not in tabela_relevante_relativa or palavra not in tabela_irrelevante_relativa:
        palavra=1
    else:
        x*=tabela_neutro_relativa[palavra]/p
        y*=tabela_relevante_relativa[palavra]/p
        z*=tabela_irrelevante_relativa[palavra]/p

In [74]:
print(x)
print(y)
print(z)

0.0219468539828806
0.038928215635872815
0.023072144642812046


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**